# Portfolio Selection Optimization
This model is an example of the classic [Markowitz portfolio selection optimization model](https://en.wikipedia.org/wiki/Markowitz_model). We want to find the fraction of the portfolio to invest among a set of stocks that balances risk and return. It is a Quadratic Programming (QP) model with vector and matrix data for returns and risk, respectively. This is best suited to a matrix formulation, so we use the Gurobi Python *matrix* interface. The basic model is fairly simple, so we also solve it parametrically to find the efficient frontier.

**Download the Repository** <br />
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip).


## Model Formulation

\begin{equation}
    \min \sum_{i=1}^{N}\sum_{i=j+1}^{N}w_{i}w_{j}\sigma_{ij} \\
\end{equation}

\begin{equation}
\sum_{i=1}^{N} \mu_{i} x_{i} \geq \mu_{0}
\end{equation}

\begin{equation}
\sum_{i=1}^{N}w_{i} = 1
\end{equation}

\begin{equation}
w^{min} \leq w_{i} \leq w^{max}

\end{equation}

### Parameters

We use the [Greek values](https://en.wikipedia.org/wiki/Greeks_\(finance\)) that are traditional in finance:

- $\delta$: n-element vector measuring the change in price for each stock
- $\sigma$: n x n matrix measuring the covariance among stocks

There is one additional parameter when solving the model parametrically:

- r: target return


### Decision Variables
- $x \ge 0$: n-element vector where each element represents the fraction of the porfolio to invest in each stock

### Objective Function
Minimize the total risk, a convex quadratic function:

\begin{equation}
\min x^t \cdot \sigma \cdot x
\end{equation}

### Constraints

Allocate the entire portfolio: the total investments should be 1.0 (100%), where $e$ is a unit vector (all 1's):

\begin{equation}
e \cdot x = 1
\end{equation}


Return: When we solve the model parametrically for different return values $r$, we add a constraint on the target return:

\begin{equation}
\delta \cdot x = r
\end{equation}

Buy-in threshold: 

\begin{equation}

\end{equation}

In [1]:
import datetime
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px
from docplex.mp.model import Model

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 300)

### Experiment Descriptor

- 1 - CPLEX 
- 2 - Restrição de retorno minimo adicionada


### Read Stock Data

In [2]:
start_date = datetime.datetime(2020,1,1)
end_date   = datetime.datetime.now()

In [3]:
assets = ['VALE3', 'GGBR4', 'CPLE6', 'BBSE3', 'ITUB4', 'WEGE3', 'JBSS3', 'PRIO3', 'MULT3', 'ITSA4', 'ABEV3', 'ALPA4',
          'ARZZ3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BEEF3', 'BHIA3', 'BPAC11', 'BRAP4','BRFS3', 'BRKM5']

tickers = (pd.Series(assets)+'.SA').tolist()

# tickers = ['AMZN', 'GOOGL', 'MSFT', 'IBM', 'META', 'NFLX']
stocks  = yf.download(tickers=tickers, start=start_date, end = end_date, rounding=True)['Adj Close']

[*********************100%%**********************]  24 of 24 completed


In [4]:
# print prices from a few stocks
# stocks_data = st_r.pivot(index='date', columns='symbol', values='close')
stocks.columns = assets
stocks.head()

,VALE3,GGBR4,CPLE6,BBSE3,ITUB4,WEGE3,JBSS3,PRIO3,MULT3,ITSA4,ABEV3,ALPA4,ARZZ3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BEEF3,BHIA3,BPAC11,BRAP4,BRFS3,BRKM5
Date,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-02,16.18,32.15,59.35,58.80,12.87,15.52,21.74,23.18,26.19,9.79,256.57,17.31,8.54,34.15,25.86,4.64,11.07,9.25,31.51,21.48,29.70,7.14,36.41,16.54
2020-01-03,15.95,32.15,58.51,56.76,12.50,15.49,21.41,23.19,25.90,10.04,251.10,17.36,8.66,34.21,27.01,4.64,10.89,9.11,31.18,21.69,29.70,7.56,36.15,16.33
2020-01-06,16.02,31.64,58.15,55.00,12.38,15.29,21.28,22.78,26.32,10.04,251.10,17.34,8.70,33.86,28.41,4.65,10.86,9.09,30.72,21.18,29.65,7.71,35.93,16.37
2020-01-07,16.06,32.36,58.42,56.82,12.78,15.17,20.92,22.38,26.34,10.49,254.82,17.41,8.74,33.47,27.67,4.67,11.03,8.97,30.00,21.56,29.73,7.72,36.19,16.50
2020-01-08,15.98,32.72,57.58,56.92,12.74,15.03,20.60,22.04,26.32,10.61,253.73,17.25,8.74,34.75,29.06,4.60,11.08,8.83,29.51,22.09,29.85,7.56,36.20,15.88


In [5]:
stocks.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1083 entries, 2020-01-02 to 2024-05-10
Data columns (total 24 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   VALE3   1083 non-null   float64
 1   GGBR4   1083 non-null   float64
 2   CPLE6   1083 non-null   float64
 3   BBSE3   1083 non-null   float64
 4   ITUB4   1083 non-null   float64
 5   WEGE3   1083 non-null   float64
 6   JBSS3   1083 non-null   float64
 7   PRIO3   1083 non-null   float64
 8   MULT3   1083 non-null   float64
 9   ITSA4   1083 non-null   float64
 10  ABEV3   1083 non-null   float64
 11  ALPA4   1083 non-null   float64
 12  ARZZ3   1083 non-null   float64
 13  AZUL4   1083 non-null   float64
 14  B3SA3   1083 non-null   float64
 15  BBAS3   1083 non-null   float64
 16  BBDC3   1083 non-null   float64
 17  BBDC4   1083 non-null   float64
 18  BEEF3   1083 non-null   float64
 19  BHIA3   1083 non-null   float64
 20  BPAC11  1083 non-null   float64
 21  BRAP4   1083 non-nu

In [6]:
# st = st.drop_duplicates(subset =['date','symbol'],keep = False)
# prices = st.pivot(index='date', columns='symbol', values='close')
prices = stocks.copy()

In [7]:
prices.head()

,VALE3,GGBR4,CPLE6,BBSE3,ITUB4,WEGE3,JBSS3,PRIO3,MULT3,ITSA4,ABEV3,ALPA4,ARZZ3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BEEF3,BHIA3,BPAC11,BRAP4,BRFS3,BRKM5
Date,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-02,16.18,32.15,59.35,58.80,12.87,15.52,21.74,23.18,26.19,9.79,256.57,17.31,8.54,34.15,25.86,4.64,11.07,9.25,31.51,21.48,29.70,7.14,36.41,16.54
2020-01-03,15.95,32.15,58.51,56.76,12.50,15.49,21.41,23.19,25.90,10.04,251.10,17.36,8.66,34.21,27.01,4.64,10.89,9.11,31.18,21.69,29.70,7.56,36.15,16.33
2020-01-06,16.02,31.64,58.15,55.00,12.38,15.29,21.28,22.78,26.32,10.04,251.10,17.34,8.70,33.86,28.41,4.65,10.86,9.09,30.72,21.18,29.65,7.71,35.93,16.37
2020-01-07,16.06,32.36,58.42,56.82,12.78,15.17,20.92,22.38,26.34,10.49,254.82,17.41,8.74,33.47,27.67,4.67,11.03,8.97,30.00,21.56,29.73,7.72,36.19,16.50
2020-01-08,15.98,32.72,57.58,56.92,12.74,15.03,20.60,22.04,26.32,10.61,253.73,17.25,8.74,34.75,29.06,4.60,11.08,8.83,29.51,22.09,29.85,7.56,36.20,15.88


In [8]:
train = prices.loc[(prices.index >= '2020-01-01') & (prices.index <= '2022-12-31')]
print(train.shape)
train.tail()

(745, 24)


,VALE3,GGBR4,CPLE6,BBSE3,ITUB4,WEGE3,JBSS3,PRIO3,MULT3,ITSA4,ABEV3,ALPA4,ARZZ3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BEEF3,BHIA3,BPAC11,BRAP4,BRFS3,BRKM5
Date,,,,,,,,,,,,,,,,,,,,,,,,
2022-12-23,13.89,14.80,78.51,11.45,12.73,14.68,12.03,13.67,29.18,11.90,56.00,23.78,24.94,7.52,23.75,7.39,20.96,7.35,23.06,19.99,21.60,35.43,77.71,37.52
2022-12-26,13.80,14.68,76.29,11.36,12.80,14.66,11.75,13.34,28.89,11.60,55.78,23.19,24.98,7.44,23.29,7.34,20.58,7.26,22.61,19.92,21.04,35.38,78.42,37.21
2022-12-27,13.80,14.45,74.36,10.74,12.35,14.17,11.83,13.28,28.84,11.76,51.84,22.65,25.58,7.19,23.93,7.27,21.60,7.18,22.48,19.94,20.32,35.52,80.29,37.04
2022-12-28,13.83,15.12,76.41,11.16,12.78,14.43,12.19,13.68,29.22,12.00,53.59,23.77,25.62,7.76,24.01,7.55,21.50,7.33,22.94,20.49,21.14,37.19,80.11,37.71
2022-12-29,13.80,15.08,75.99,11.01,12.60,14.45,12.13,13.67,29.29,12.22,52.50,23.23,25.35,8.28,23.76,7.58,21.44,7.29,22.89,20.82,20.94,37.15,80.01,37.53


In [9]:
oos = prices.loc[(prices.index >= '2022-12-31')]
print(oos.shape)
oos.tail()

(338, 24)


,VALE3,GGBR4,CPLE6,BBSE3,ITUB4,WEGE3,JBSS3,PRIO3,MULT3,ITSA4,ABEV3,ALPA4,ARZZ3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BEEF3,BHIA3,BPAC11,BRAP4,BRFS3,BRKM5
Date,,,,,,,,,,,,,,,,,,,,,,,,
2024-05-06,12.45,9.78,51.75,10.93,11.51,28.38,12.32,13.77,33.26,6.01,6.97,35.15,20.30,16.18,19.70,9.55,19.64,9.83,32.40,23.88,24.41,47.19,64.18,38.63
2024-05-07,12.60,9.68,51.79,10.96,11.58,28.29,12.36,13.77,32.80,6.02,7.03,35.32,20.33,16.65,20.05,9.56,19.31,9.98,33.07,24.31,24.07,47.17,64.58,38.90
2024-05-08,12.17,9.80,51.98,11.12,11.67,28.38,12.28,13.80,32.62,6.08,6.96,34.61,20.21,18.51,19.71,9.54,19.45,10.09,33.28,24.47,23.96,46.54,63.99,39.09
2024-05-09,12.05,9.96,49.75,11.08,11.21,27.14,11.96,13.45,32.62,6.24,7.12,33.21,20.28,18.91,19.28,9.24,19.35,9.93,32.28,24.61,23.44,46.78,64.51,38.90
2024-05-10,12.04,10.29,47.82,11.03,10.94,27.62,11.95,13.37,33.13,6.34,6.81,33.52,20.10,18.32,19.06,9.13,19.14,10.04,32.65,24.62,23.65,47.00,64.29,38.97


In [10]:
train.shape, oos.shape

((745, 24), (338, 24))

In [11]:
daily_returns = train.pct_change().head()
daily_returns = daily_returns.iloc[1:] # deletes the first row that has no daily return
daily_returns = daily_returns.dropna(axis='columns') # filter out stocks with NaNs
daily_returns.head()

,VALE3,GGBR4,CPLE6,BBSE3,ITUB4,WEGE3,JBSS3,PRIO3,MULT3,ITSA4,ABEV3,ALPA4,ARZZ3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BEEF3,BHIA3,BPAC11,BRAP4,BRFS3,BRKM5
Date,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-03,-0.014215,0.000000,-0.014153,-0.034694,-0.028749,-0.001933,-0.015179,0.000431,-0.011073,0.025536,-0.021320,0.002889,0.014052,0.001757,0.044470,0.000000,-0.016260,-0.015135,-0.010473,0.009777,0.000000,0.058824,-0.007141,-0.012696
2020-01-06,0.004389,-0.015863,-0.006153,-0.031008,-0.009600,-0.012912,-0.006072,-0.017680,0.016216,0.000000,0.000000,-0.001152,0.004619,-0.010231,0.051833,0.002155,-0.002755,-0.002195,-0.014753,-0.023513,-0.001684,0.019841,-0.006086,0.002449
2020-01-07,0.002497,0.022756,0.004643,0.033091,0.032310,-0.007848,-0.016917,-0.017559,0.000760,0.044821,0.014815,0.004037,0.004598,-0.011518,-0.026047,0.004301,0.015654,-0.013201,-0.023438,0.017941,0.002698,0.001297,0.007236,0.007941
2020-01-08,-0.004981,0.011125,-0.014379,0.001760,-0.003130,-0.009229,-0.015296,-0.015192,-0.000759,0.011439,-0.004278,-0.009190,0.000000,0.038243,0.050235,-0.014989,0.004533,-0.015608,-0.016333,0.024583,0.004036,-0.020725,0.000276,-0.037576


In [12]:
mu = daily_returns.mean()
mu

VALE3    -0.003078
GGBR4     0.004504
CPLE6    -0.007510
BBSE3    -0.007713
ITUB4    -0.002292
WEGE3    -0.007980
JBSS3    -0.013366
PRIO3    -0.012500
MULT3     0.001286
ITSA4     0.020449
ABEV3    -0.002696
ALPA4    -0.000854
ARZZ3     0.005817
AZUL4     0.004563
B3SA3     0.030123
BBAS3    -0.002133
BBDC3     0.000293
BBDC4    -0.011535
BEEF3    -0.016249
BHIA3     0.007197
BPAC11    0.001263
BRAP4     0.014809
BRFS3    -0.001429
BRKM5    -0.009970
dtype: float64

In [13]:
mu.to_dict()

{'VALE3': -0.003077701690250928,
 'GGBR4': 0.004504427255260346,
 'CPLE6': -0.0075103988333856675,
 'BBSE3': -0.007712694179073415,
 'ITUB4': -0.0022921853742213505,
 'WEGE3': -0.007980388327582827,
 'JBSS3': -0.013366245543829508,
 'PRIO3': -0.012500006614838965,
 'MULT3': 0.0012859661485736251,
 'ITSA4': 0.020449111195259495,
 'ABEV3': -0.00269560805932742,
 'ALPA4': -0.0008541954291993725,
 'ARZZ3': 0.0058170402605026705,
 'AZUL4': 0.004562803873312954,
 'B3SA3': 0.03012265595569219,
 'BBAS3': -0.002133261419818566,
 'BBDC3': 0.0002929708381170515,
 'BBDC4': -0.011534856443447233,
 'BEEF3': -0.01624918647878093,
 'BHIA3': 0.007196852779015134,
 'BPAC11': 0.0012627425710689533,
 'BRAP4': 0.01480910687830439,
 'BRFS3': -0.0014285092363048313,
 'BRKM5': -0.009970353825081213}

In [14]:
sigma = daily_returns.cov()
sigma.head()

,VALE3,GGBR4,CPLE6,BBSE3,ITUB4,WEGE3,JBSS3,PRIO3,MULT3,ITSA4,ABEV3,ALPA4,ARZZ3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BEEF3,BHIA3,BPAC11,BRAP4,BRFS3,BRKM5
VALE3,0.000071,-0.000004,0.000055,0.000112,0.000145,-0.000034,0.000020,-0.000069,0.000083,-0.000019,0.000109,-2.578887e-07,-0.000032,-0.000078,-0.000116,0.000023,0.000080,0.000036,-0.000031,-0.000077,-1.737441e-06,-0.000153,0.000025,0.000092
GGBR4,-0.000004,0.000271,0.000060,0.000468,0.000298,0.000022,-0.000073,-0.000021,-0.000091,0.000260,0.000113,7.763800e-06,-0.000024,0.000081,-0.000466,-0.000022,0.000148,-0.000077,-0.000063,0.000308,3.675214e-05,-0.000261,0.000097,-0.000032
CPLE6,0.000055,0.000060,0.000080,0.000193,0.000197,-0.000012,-0.000003,-0.000045,0.000037,0.000099,0.000117,3.047686e-05,-0.000010,-0.000143,-0.000296,0.000053,0.000088,0.000015,-0.000041,-0.000016,9.280299e-07,-0.000069,0.000042,0.000147
BBSE3,0.000112,0.000468,0.000193,0.001008,0.000763,-0.000018,-0.000095,-0.000153,-0.000018,0.000416,0.000380,8.856040e-06,-0.000100,0.000028,-0.000998,-0.000006,0.000395,-0.000076,-0.000162,0.000416,6.251532e-05,-0.000731,0.000211,0.000085
ITUB4,0.000145,0.000298,0.000197,0.000763,0.000650,-0.000039,-0.000042,-0.000159,0.000067,0.000289,0.000360,2.646122e-05,-0.000082,-0.000134,-0.000833,0.000049,0.000329,-0.000009,-0.000137,0.000171,3.409455e-05,-0.000546,0.000161,0.000208


### Decision Optimization with CPLEX (Community Edition)
    

In [15]:
keys = list(assets)
keys

['VALE3',
 'GGBR4',
 'CPLE6',
 'BBSE3',
 'ITUB4',
 'WEGE3',
 'JBSS3',
 'PRIO3',
 'MULT3',
 'ITSA4',
 'ABEV3',
 'ALPA4',
 'ARZZ3',
 'AZUL4',
 'B3SA3',
 'BBAS3',
 'BBDC3',
 'BBDC4',
 'BEEF3',
 'BHIA3',
 'BPAC11',
 'BRAP4',
 'BRFS3',
 'BRKM5']

In [16]:
model = Model()
x = model.continuous_var_dict(keys, name='asset')

In [17]:
x

{'VALE3': docplex.mp.Var(type=C,name='asset_VALE3'),
 'GGBR4': docplex.mp.Var(type=C,name='asset_GGBR4'),
 'CPLE6': docplex.mp.Var(type=C,name='asset_CPLE6'),
 'BBSE3': docplex.mp.Var(type=C,name='asset_BBSE3'),
 'ITUB4': docplex.mp.Var(type=C,name='asset_ITUB4'),
 'WEGE3': docplex.mp.Var(type=C,name='asset_WEGE3'),
 'JBSS3': docplex.mp.Var(type=C,name='asset_JBSS3'),
 'PRIO3': docplex.mp.Var(type=C,name='asset_PRIO3'),
 'MULT3': docplex.mp.Var(type=C,name='asset_MULT3'),
 'ITSA4': docplex.mp.Var(type=C,name='asset_ITSA4'),
 'ABEV3': docplex.mp.Var(type=C,name='asset_ABEV3'),
 'ALPA4': docplex.mp.Var(type=C,name='asset_ALPA4'),
 'ARZZ3': docplex.mp.Var(type=C,name='asset_ARZZ3'),
 'AZUL4': docplex.mp.Var(type=C,name='asset_AZUL4'),
 'B3SA3': docplex.mp.Var(type=C,name='asset_B3SA3'),
 'BBAS3': docplex.mp.Var(type=C,name='asset_BBAS3'),
 'BBDC3': docplex.mp.Var(type=C,name='asset_BBDC3'),
 'BBDC4': docplex.mp.Var(type=C,name='asset_BBDC4'),
 'BEEF3': docplex.mp.Var(type=C,name='asset_BE

In [18]:
mu.to_dict()

{'VALE3': -0.003077701690250928,
 'GGBR4': 0.004504427255260346,
 'CPLE6': -0.0075103988333856675,
 'BBSE3': -0.007712694179073415,
 'ITUB4': -0.0022921853742213505,
 'WEGE3': -0.007980388327582827,
 'JBSS3': -0.013366245543829508,
 'PRIO3': -0.012500006614838965,
 'MULT3': 0.0012859661485736251,
 'ITSA4': 0.020449111195259495,
 'ABEV3': -0.00269560805932742,
 'ALPA4': -0.0008541954291993725,
 'ARZZ3': 0.0058170402605026705,
 'AZUL4': 0.004562803873312954,
 'B3SA3': 0.03012265595569219,
 'BBAS3': -0.002133261419818566,
 'BBDC3': 0.0002929708381170515,
 'BBDC4': -0.011534856443447233,
 'BEEF3': -0.01624918647878093,
 'BHIA3': 0.007196852779015134,
 'BPAC11': 0.0012627425710689533,
 'BRAP4': 0.01480910687830439,
 'BRFS3': -0.0014285092363048313,
 'BRKM5': -0.009970353825081213}

In [19]:
c1 = model.add_constraint(model.sum(x[i] for i in keys) == 1) 
c2 = model.add_constraints(x[i] <= 0.20 for i in keys) 
c3 = model.add_constraints(x[i] >= 0.01 for i in keys)
c4 = model.add_constraint(model.sum(x[i] * mu.to_dict()[i] for i in keys) >= .01)

In [20]:
# c1, c2, c3, c4

In [21]:
variance = model.sum(float(sigma[i][j]) * x[i] * x[j] for i in sigma.index for j in sigma.index)
model.minimize(variance)

In [22]:
solution = model.solve(log_output=True)

Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Number of nonzeros in lower triangle of Q = 276
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.01 ticks)
Summary statistics for factor of Q:
  Rows in Factor            = 24
  Integer space required    = 24
  Total non-zeros in factor = 300
  Total FP ops to factor    = 4900
Tried aggregator 1 time.
QP Presolve eliminated 69 rows and 21 columns.
QP Presolve added 0 rows and 24 columns.
Reduced QP has 5 rows, 27 columns, and 120 nonzeros.
Reduced QP objective Q matrix has 3 nonzeros.
Presolve time = 0.02 sec. (0.05 ticks)
Parallel mode: using up to 16 threads for barrier.
Number of nonzeros in lower triangle of A*A' = 10
Using Approximate Minimum Degree ordering
Total time for automatic ordering = 0.00 sec. (0.00 ticks)
Summary statistics for Cholesky factor:
  Threads                   = 16
  Rows in Factor            = 5
  Integer space r

In [23]:
model.print_solution()

objective: 0.000
status: OPTIMAL_SOLUTION(2)
  asset_VALE3=0.014
  asset_GGBR4=0.016
  asset_CPLE6=0.011
  asset_BBSE3=0.011
  asset_ITUB4=0.165
  asset_WEGE3=0.011
  asset_JBSS3=0.010
  asset_PRIO3=0.010
  asset_MULT3=0.109
  asset_ITSA4=0.194
  asset_ABEV3=0.016
  asset_ALPA4=0.014
  asset_ARZZ3=0.024
  asset_AZUL4=0.017
  asset_B3SA3=0.199
  asset_BBAS3=0.013
  asset_BBDC3=0.021
  asset_BBDC4=0.011
  asset_BEEF3=0.010
  asset_BHIA3=0.017
  asset_BPAC11=0.016
  asset_BRAP4=0.063
  asset_BRFS3=0.014
  asset_BRKM5=0.011


In [24]:
solution_dict = {}
for i in keys:
   solution_dict[i] = x[i].solution_value
   print(f"{i} - {x[i].solution_value}")

VALE3 - 0.014280660110468352
GGBR4 - 0.016124165342481334
CPLE6 - 0.011486730464761194
BBSE3 - 0.010958217157922044
ITUB4 - 0.16527385566406433
WEGE3 - 0.011134729079849607
JBSS3 - 0.010481883637324356
PRIO3 - 0.010477003642375188
MULT3 - 0.10926684197493314
ITSA4 - 0.19371716472358408
ABEV3 - 0.016070530101618814
ALPA4 - 0.014117098203111904
ARZZ3 - 0.023985441398626177
AZUL4 - 0.0166244587843351
B3SA3 - 0.1993794571051403
BBAS3 - 0.01348812958684708
BBDC3 - 0.020538649926454194
BBDC4 - 0.010711515655371623
BEEF3 - 0.010262073300551762
BHIA3 - 0.017361833244866013
BPAC11 - 0.01605085130727069
BRAP4 - 0.06314603217565976
BRFS3 - 0.014032276661424987
BRKM5 - 0.011030400750957394


In [25]:
solution_dict

{'VALE3': 0.014280660110468352,
 'GGBR4': 0.016124165342481334,
 'CPLE6': 0.011486730464761194,
 'BBSE3': 0.010958217157922044,
 'ITUB4': 0.16527385566406433,
 'WEGE3': 0.011134729079849607,
 'JBSS3': 0.010481883637324356,
 'PRIO3': 0.010477003642375188,
 'MULT3': 0.10926684197493314,
 'ITSA4': 0.19371716472358408,
 'ABEV3': 0.016070530101618814,
 'ALPA4': 0.014117098203111904,
 'ARZZ3': 0.023985441398626177,
 'AZUL4': 0.0166244587843351,
 'B3SA3': 0.1993794571051403,
 'BBAS3': 0.01348812958684708,
 'BBDC3': 0.020538649926454194,
 'BBDC4': 0.010711515655371623,
 'BEEF3': 0.010262073300551762,
 'BHIA3': 0.017361833244866013,
 'BPAC11': 0.01605085130727069,
 'BRAP4': 0.06314603217565976,
 'BRFS3': 0.014032276661424987,
 'BRKM5': 0.011030400750957394}

In [26]:
model.report()

* model docplex_model1 solved with objective = 0.000


In [27]:
total_return = 0
for i in keys:
    total_return += x[i].solution_value * mu[i]

total_return

0.010049784806436498

In [28]:
print(f'Variância encontrada pela função objetivo: {model.objective_value}')
print(f'Retorno total: {total_return}')

Variância encontrada pela função objetivo: 2.155908868875697e-13
Retorno total: 0.010049784806436498


In [29]:
delta = mu
std = daily_returns.std()
std.head()

VALE3    0.008456
GGBR4    0.016271
CPLE6    0.008960
BBSE3    0.031756
ITUB4    0.025258
dtype: float64

In [30]:
mu.min()

-0.016233973357476783

### Test OOS

In [31]:
oos.shape

(273, 24)

In [32]:
oos.head(7)

,VALE3,GGBR4,CPLE6,BBSE3,ITUB4,WEGE3,JBSS3,PRIO3,MULT3,ITSA4,ABEV3,ALPA4,ARZZ3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BEEF3,BHIA3,BPAC11,BRAP4,BRFS3,BRKM5
Date,,,,,,,,,,,,,,,,,,,,,,,,
2023-01-02,13.49,14.33,71.98,10.30,11.86,30.70,11.93,13.37,30.11,11.98,50.53,21.73,26.41,7.96,23.22,7.35,25.76,7.42,23.38,20.31,19.93,36.70,84.08,37.49
2023-01-03,13.47,13.40,69.74,10.16,11.46,30.12,11.37,12.70,29.17,11.64,48.12,20.69,26.21,7.56,22.46,7.22,25.81,7.23,22.89,19.87,19.29,34.24,83.93,36.56
2023-01-04,13.54,13.49,71.34,10.56,11.55,30.51,11.47,12.74,29.32,12.08,49.87,21.35,26.46,7.71,22.41,7.42,25.69,7.29,22.96,20.17,19.62,34.04,84.08,36.80
2023-01-05,13.58,13.86,72.19,11.55,11.90,31.90,11.84,13.29,29.38,12.33,51.18,22.21,27.12,8.38,23.09,7.36,26.32,7.40,23.67,20.92,20.14,34.91,85.49,36.88
2023-01-06,13.61,13.87,73.61,11.84,12.19,32.35,12.15,13.67,30.24,12.51,52.71,22.34,27.21,8.64,23.23,7.26,26.52,7.52,24.08,21.03,20.68,35.12,86.85,37.42
2023-01-09,13.68,14.22,72.86,12.00,12.02,32.03,12.18,13.84,30.38,12.59,53.15,22.13,27.55,8.69,23.39,7.19,27.06,7.58,24.25,21.66,20.42,35.95,86.94,37.31
2023-01-10,13.82,14.78,74.84,12.34,12.45,32.58,12.44,14.25,30.84,12.68,56.65,23.32,28.12,9.06,23.05,7.16,27.81,7.74,24.72,21.44,21.04,36.15,88.01,36.95


In [33]:
oos_returns = oos.pct_change()
oos_returns = oos_returns.iloc[1:]               # deletes the first row that has no daily return
oos_returns = oos_returns.dropna(axis='columns') # filter out stocks with NaNs
print(oos_returns.shape)
oos_returns.head(7)

(272, 24)


,VALE3,GGBR4,CPLE6,BBSE3,ITUB4,WEGE3,JBSS3,PRIO3,MULT3,ITSA4,ABEV3,ALPA4,ARZZ3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BEEF3,BHIA3,BPAC11,BRAP4,BRFS3,BRKM5
Date,,,,,,,,,,,,,,,,,,,,,,,,
2023-01-03,-0.001483,-0.064899,-0.031120,-0.013592,-0.033727,-0.018893,-0.046940,-0.050112,-0.031219,-0.028381,-0.047694,-0.047860,-0.007573,-0.050251,-0.032730,-0.017687,0.001941,-0.025606,-0.020958,-0.021664,-0.032112,-0.067030,-0.001784,-0.024807
2023-01-04,0.005197,0.006716,0.022942,0.039370,0.007853,0.012948,0.008795,0.003150,0.005142,0.037801,0.036367,0.031899,0.009538,0.019841,-0.002226,0.027701,-0.004649,0.008299,0.003058,0.015098,0.017107,-0.005841,0.001787,0.006565
2023-01-05,0.002954,0.027428,0.011915,0.093750,0.030303,0.045559,0.032258,0.043171,0.002046,0.020695,0.026268,0.040281,0.024943,0.086900,0.030344,-0.008086,0.024523,0.015089,0.030923,0.037184,0.026504,0.025558,0.016770,0.002174
2023-01-06,0.002209,0.000722,0.019670,0.025108,0.024370,0.014107,0.026182,0.028593,0.029272,0.014599,0.029894,0.005853,0.003319,0.031026,0.006063,-0.013587,0.007599,0.016216,0.017322,0.005258,0.026812,0.006015,0.015908,0.014642
2023-01-09,0.005143,0.025234,-0.010189,0.013514,-0.013946,-0.009892,0.002469,0.012436,0.004630,0.006395,0.008348,-0.009400,0.012495,0.005787,0.006888,-0.009642,0.020362,0.007979,0.007060,0.029957,-0.012573,0.023633,0.001036,-0.002940
2023-01-10,0.010234,0.039381,0.027175,0.028333,0.035774,0.017171,0.021346,0.029624,0.015142,0.007149,0.065851,0.053773,0.020690,0.042578,-0.014536,-0.004172,0.027716,0.021108,0.019381,-0.010157,0.030362,0.005563,0.012307,-0.009649
2023-01-11,-0.017366,0.023681,0.025120,0.000810,0.045783,0.014119,0.010450,0.020351,0.005837,0.077287,0.003883,-0.018439,0.001422,-0.066225,0.030803,0.047486,0.010068,0.007752,0.017395,-0.010261,0.038973,0.077732,-0.003522,0.027876


No geral, para um portfolio composto por $n$ ativos com proporções de investimento $w_{i}$ tal que $w_{1} + . . . + w_{n} = 1$. O retorno total e a taxa de retorno são definidos como:

\begin{equation}
    R_{p,t} = \sum_{i=1}^{N}w_{i}R_{i,t} \\
\end{equation}


In [34]:
solution_dict

{'VALE3': 0.014280660110468352,
 'GGBR4': 0.016124165342481334,
 'CPLE6': 0.011486730464761194,
 'BBSE3': 0.010958217157922044,
 'ITUB4': 0.16527385566406433,
 'WEGE3': 0.011134729079849607,
 'JBSS3': 0.010481883637324356,
 'PRIO3': 0.010477003642375188,
 'MULT3': 0.10926684197493314,
 'ITSA4': 0.19371716472358408,
 'ABEV3': 0.016070530101618814,
 'ALPA4': 0.014117098203111904,
 'ARZZ3': 0.023985441398626177,
 'AZUL4': 0.0166244587843351,
 'B3SA3': 0.1993794571051403,
 'BBAS3': 0.01348812958684708,
 'BBDC3': 0.020538649926454194,
 'BBDC4': 0.010711515655371623,
 'BEEF3': 0.010262073300551762,
 'BHIA3': 0.017361833244866013,
 'BPAC11': 0.01605085130727069,
 'BRAP4': 0.06314603217565976,
 'BRFS3': 0.014032276661424987,
 'BRKM5': 0.011030400750957394}

In [35]:
# Get seven days
seven_days_oos_returns = oos_returns.iloc[:7,:]
seven_days_oos_returns.head(10)

,VALE3,GGBR4,CPLE6,BBSE3,ITUB4,WEGE3,JBSS3,PRIO3,MULT3,ITSA4,ABEV3,ALPA4,ARZZ3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BEEF3,BHIA3,BPAC11,BRAP4,BRFS3,BRKM5
Date,,,,,,,,,,,,,,,,,,,,,,,,
2023-01-03,-0.001483,-0.064899,-0.031120,-0.013592,-0.033727,-0.018893,-0.046940,-0.050112,-0.031219,-0.028381,-0.047694,-0.047860,-0.007573,-0.050251,-0.032730,-0.017687,0.001941,-0.025606,-0.020958,-0.021664,-0.032112,-0.067030,-0.001784,-0.024807
2023-01-04,0.005197,0.006716,0.022942,0.039370,0.007853,0.012948,0.008795,0.003150,0.005142,0.037801,0.036367,0.031899,0.009538,0.019841,-0.002226,0.027701,-0.004649,0.008299,0.003058,0.015098,0.017107,-0.005841,0.001787,0.006565
2023-01-05,0.002954,0.027428,0.011915,0.093750,0.030303,0.045559,0.032258,0.043171,0.002046,0.020695,0.026268,0.040281,0.024943,0.086900,0.030344,-0.008086,0.024523,0.015089,0.030923,0.037184,0.026504,0.025558,0.016770,0.002174
2023-01-06,0.002209,0.000722,0.019670,0.025108,0.024370,0.014107,0.026182,0.028593,0.029272,0.014599,0.029894,0.005853,0.003319,0.031026,0.006063,-0.013587,0.007599,0.016216,0.017322,0.005258,0.026812,0.006015,0.015908,0.014642
2023-01-09,0.005143,0.025234,-0.010189,0.013514,-0.013946,-0.009892,0.002469,0.012436,0.004630,0.006395,0.008348,-0.009400,0.012495,0.005787,0.006888,-0.009642,0.020362,0.007979,0.007060,0.029957,-0.012573,0.023633,0.001036,-0.002940
2023-01-10,0.010234,0.039381,0.027175,0.028333,0.035774,0.017171,0.021346,0.029624,0.015142,0.007149,0.065851,0.053773,0.020690,0.042578,-0.014536,-0.004172,0.027716,0.021108,0.019381,-0.010157,0.030362,0.005563,0.012307,-0.009649
2023-01-11,-0.017366,0.023681,0.025120,0.000810,0.045783,0.014119,0.010450,0.020351,0.005837,0.077287,0.003883,-0.018439,0.001422,-0.066225,0.030803,0.047486,0.010068,0.007752,0.017395,-0.010261,0.038973,0.077732,-0.003522,0.027876


In [36]:
weighted_ret = pd.DataFrame()

for col in seven_days_oos_returns.columns:
    print(f"{col} - {solution_dict[col]}")
    weighted_ret[col] = (seven_days_oos_returns[col] * solution_dict[col])

VALE3 - 0.014280660110468352
GGBR4 - 0.016124165342481334
CPLE6 - 0.011486730464761194
BBSE3 - 0.010958217157922044
ITUB4 - 0.16527385566406433
WEGE3 - 0.011134729079849607
JBSS3 - 0.010481883637324356
PRIO3 - 0.010477003642375188
MULT3 - 0.10926684197493314
ITSA4 - 0.19371716472358408
ABEV3 - 0.016070530101618814
ALPA4 - 0.014117098203111904
ARZZ3 - 0.023985441398626177
AZUL4 - 0.0166244587843351
B3SA3 - 0.1993794571051403
BBAS3 - 0.01348812958684708
BBDC3 - 0.020538649926454194
BBDC4 - 0.010711515655371623
BEEF3 - 0.010262073300551762
BHIA3 - 0.017361833244866013
BPAC11 - 0.01605085130727069
BRAP4 - 0.06314603217565976
BRFS3 - 0.014032276661424987
BRKM5 - 0.011030400750957394


In [37]:
weighted_ret.head()

,VALE3,GGBR4,CPLE6,BBSE3,ITUB4,WEGE3,JBSS3,PRIO3,MULT3,ITSA4,ABEV3,ALPA4,ARZZ3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BEEF3,BHIA3,BPAC11,BRAP4,BRFS3,BRKM5
Date,,,,,,,,,,,,,,,,,,,,,,,,
2023-01-03,-0.000021,-0.001046,-0.000357,-0.000149,-0.005574,-0.000210,-0.000492,-0.000525,-0.003411,-0.005498,-0.000766,-0.000676,-0.000182,-0.000835,-0.006526,-0.000239,0.000040,-0.000274,-0.000215,-0.000376,-0.000515,-0.004233,-0.000025,-0.000274
2023-01-04,0.000074,0.000108,0.000264,0.000431,0.001298,0.000144,0.000092,0.000033,0.000562,0.007323,0.000584,0.000450,0.000229,0.000330,-0.000444,0.000374,-0.000095,0.000089,0.000031,0.000262,0.000275,-0.000369,0.000025,0.000072
2023-01-05,0.000042,0.000442,0.000137,0.001027,0.005008,0.000507,0.000338,0.000452,0.000224,0.004009,0.000422,0.000569,0.000598,0.001445,0.006050,-0.000109,0.000504,0.000162,0.000317,0.000646,0.000425,0.001614,0.000235,0.000024
2023-01-06,0.000032,0.000012,0.000226,0.000275,0.004028,0.000157,0.000274,0.000300,0.003198,0.002828,0.000480,0.000083,0.000080,0.000516,0.001209,-0.000183,0.000156,0.000174,0.000178,0.000091,0.000430,0.000380,0.000223,0.000162
2023-01-09,0.000073,0.000407,-0.000117,0.000148,-0.002305,-0.000110,0.000026,0.000130,0.000506,0.001239,0.000134,-0.000133,0.000300,0.000096,0.001373,-0.000130,0.000418,0.000085,0.000072,0.000520,-0.000202,0.001492,0.000015,-0.000032


In [38]:
weighted_ret['portfolio_return'] = weighted_ret.sum(axis=1)

weighted_ret.head()

,VALE3,GGBR4,CPLE6,BBSE3,ITUB4,WEGE3,JBSS3,PRIO3,MULT3,ITSA4,ABEV3,ALPA4,ARZZ3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BEEF3,BHIA3,BPAC11,BRAP4,BRFS3,BRKM5,portfolio_return
Date,,,,,,,,,,,,,,,,,,,,,,,,,
2023-01-03,-0.000021,-0.001046,-0.000357,-0.000149,-0.005574,-0.000210,-0.000492,-0.000525,-0.003411,-0.005498,-0.000766,-0.000676,-0.000182,-0.000835,-0.006526,-0.000239,0.000040,-0.000274,-0.000215,-0.000376,-0.000515,-0.004233,-0.000025,-0.000274,-0.032380
2023-01-04,0.000074,0.000108,0.000264,0.000431,0.001298,0.000144,0.000092,0.000033,0.000562,0.007323,0.000584,0.000450,0.000229,0.000330,-0.000444,0.000374,-0.000095,0.000089,0.000031,0.000262,0.000275,-0.000369,0.000025,0.000072,0.012143
2023-01-05,0.000042,0.000442,0.000137,0.001027,0.005008,0.000507,0.000338,0.000452,0.000224,0.004009,0.000422,0.000569,0.000598,0.001445,0.006050,-0.000109,0.000504,0.000162,0.000317,0.000646,0.000425,0.001614,0.000235,0.000024,0.025089
2023-01-06,0.000032,0.000012,0.000226,0.000275,0.004028,0.000157,0.000274,0.000300,0.003198,0.002828,0.000480,0.000083,0.000080,0.000516,0.001209,-0.000183,0.000156,0.000174,0.000178,0.000091,0.000430,0.000380,0.000223,0.000162,0.015307
2023-01-09,0.000073,0.000407,-0.000117,0.000148,-0.002305,-0.000110,0.000026,0.000130,0.000506,0.001239,0.000134,-0.000133,0.000300,0.000096,0.001373,-0.000130,0.000418,0.000085,0.000072,0.000520,-0.000202,0.001492,0.000015,-0.000032,0.004007


Retorno cumulativo simples por dia do portfólio é dado por:

\begin{equation}
    R_{cum} = (1 + R_{i,t_{1}}) * (1 + R_{i,t_{2}}) *...* (1 + R_{i,t_{n}})\\
\end{equation}


In [39]:
weighted_ret['cumulative_return'] = (weighted_ret['portfolio_return'] + 1).cumprod()-1
weighted_ret.head()

,VALE3,GGBR4,CPLE6,BBSE3,ITUB4,WEGE3,JBSS3,PRIO3,MULT3,ITSA4,ABEV3,ALPA4,ARZZ3,AZUL4,B3SA3,BBAS3,BBDC3,BBDC4,BEEF3,BHIA3,BPAC11,BRAP4,BRFS3,BRKM5,portfolio_return,cumulative_return
Date,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-01-03,-0.000021,-0.001046,-0.000357,-0.000149,-0.005574,-0.000210,-0.000492,-0.000525,-0.003411,-0.005498,-0.000766,-0.000676,-0.000182,-0.000835,-0.006526,-0.000239,0.000040,-0.000274,-0.000215,-0.000376,-0.000515,-0.004233,-0.000025,-0.000274,-0.032380,-0.032380
2023-01-04,0.000074,0.000108,0.000264,0.000431,0.001298,0.000144,0.000092,0.000033,0.000562,0.007323,0.000584,0.000450,0.000229,0.000330,-0.000444,0.000374,-0.000095,0.000089,0.000031,0.000262,0.000275,-0.000369,0.000025,0.000072,0.012143,-0.020631
2023-01-05,0.000042,0.000442,0.000137,0.001027,0.005008,0.000507,0.000338,0.000452,0.000224,0.004009,0.000422,0.000569,0.000598,0.001445,0.006050,-0.000109,0.000504,0.000162,0.000317,0.000646,0.000425,0.001614,0.000235,0.000024,0.025089,0.003940
2023-01-06,0.000032,0.000012,0.000226,0.000275,0.004028,0.000157,0.000274,0.000300,0.003198,0.002828,0.000480,0.000083,0.000080,0.000516,0.001209,-0.000183,0.000156,0.000174,0.000178,0.000091,0.000430,0.000380,0.000223,0.000162,0.015307,0.019308
2023-01-09,0.000073,0.000407,-0.000117,0.000148,-0.002305,-0.000110,0.000026,0.000130,0.000506,0.001239,0.000134,-0.000133,0.000300,0.000096,0.001373,-0.000130,0.000418,0.000085,0.000072,0.000520,-0.000202,0.001492,0.000015,-0.000032,0.004007,0.023392


In [40]:
weighted_ret[['portfolio_return', 'cumulative_return']]

,portfolio_return,cumulative_return
Date,,
2023-01-03,-0.032380,-0.032380
2023-01-04,0.012143,-0.020631
2023-01-05,0.025089,0.003940
2023-01-06,0.015307,0.019308
2023-01-09,0.004007,0.023392
2023-01-10,0.012871,0.036563
2023-01-11,0.035687,0.073555


In [45]:
fig = px.line(weighted_ret['portfolio_return'], facet_col_wrap=1)
fig.update_layout(
    title={
        'text': "Portfolio Returns OOS"})
fig.update_yaxes(matches=None)

In [46]:
fig = px.line(weighted_ret['cumulative_return'], facet_col_wrap=1)
fig.update_layout(
    title={
        'text': "Portfolio Cumulative Returns OOS"})
fig.update_yaxes(matches=None)
fig.show()




#### Benchmark ?? - Sharpe Ratio (Porém sozinho não significa muita coisa)

- https://ibmdecisionoptimization.github.io/tutorials/html/Beyond_Linear_Programming.html